# Avance 3: Transformación de Datos con Python

## 1. Configuración, Carga y Unificación de Datos
El primer paso es cargar todas las fuentes de datos (CSV) y unificarlas en un único DataFrame principal para realizar las operaciones de Feature Engineering.

In [11]:
# 1 Configuración de Entorno y Carga de Archivos
import pandas as pd
import numpy as np

# 1. Cargar los archivos CSV
try:
    df_sales = pd.read_csv('sales.csv')
    df_products = pd.read_csv('products.csv')
    df_employees = pd.read_csv('employees.csv')
    df_customers = pd.read_csv('customers.csv')
    df_categories = pd.read_csv('categories.csv')
    df_cities = pd.read_csv('cities.csv')
    df_countries = pd.read_csv('countries.csv')
except FileNotFoundError:
    print("¡Advertencia! Asegúrate de que los archivos CSV estén cargados en el entorno de Colab.")


In [12]:
# 2. Renombrado y estandarización de columnas

df_products.rename(columns={'ProductID': 'product_id', 'Price': 'unit_price', 'CategoryID': 'category_id'}, inplace=True)
df_categories.rename(columns={'CategoryID': 'category_id', 'CategoryName': 'category_name'}, inplace=True)
df_employees.rename(columns={'EmployeeID': 'employee_id', 'BirthDate': 'birth_date', 'HireDate': 'hire_date'}, inplace=True)
df_sales.rename(columns={'SalesPersonID': 'sales_person_id', 'ProductID': 'product_id', 'Quantity': 'quantity', 'Discount': 'discount', 'SalesDate': 'sales_date'}, inplace=True)
df_customers.rename(columns={'CustomerID': 'customer_id', 'CityID': 'city_id'}, inplace=True)
df_cities.rename(columns={'CityID': 'city_id', 'CityName': 'city_name'}, inplace=True)
df_countries.rename(columns={'CountryID': 'country_id', 'CountryName': 'country_name'}, inplace=True)

In [13]:
# --- 3. UNIFICACIÓN DEL DATASET BASE ---
df_data = df_sales.copy()

# A. Unir con Products y Categories
df_data = pd.merge(df_data, df_products[['product_id', 'unit_price', 'category_id', 'ProductName']], on='product_id', how='left')
df_data = pd.merge(df_data, df_categories[['category_id', 'category_name']], on='category_id', how='left')

# B. Unir con Employees (para las Features de Empleados)
df_data = pd.merge(df_data, df_employees[['employee_id', 'birth_date', 'hire_date']], left_on='sales_person_id', right_on='employee_id', how='left', suffixes=('_sale', '_emp'))
df_data.drop(columns=['employee_id'], inplace=True)

# C. Conversión de fechas a formato datetime
date_cols = ['sales_date', 'birth_date', 'hire_date']
for col in date_cols:
    df_data[col] = pd.to_datetime(df_data[col], errors='coerce')



print("DataFrame Base Unificado y Columnas Estandarizadas (snake_case).")
print("Columnas de fechas verificadas:")
print(df_data[date_cols].dtypes)

DataFrame Base Unificado y Columnas Estandarizadas (snake_case).
Columnas de fechas verificadas:
sales_date    datetime64[ns]
birth_date    datetime64[ns]
hire_date     datetime64[ns]
dtype: object


## 2. Cálculo de TotalPriceCalculated

In [14]:
# Fórmula: $TotalPriceCalculated=(Quantity \times UnitPrice) \times (1-Discount)$

# Justificación: El campo 'total_price' en el dataset de origen es inconsistente o nulo.
# Se utiliza la información de 'unit_price' de la tabla 'products' para obtener el valor real.

df_data['TotalPriceCalculated'] = (
    df_data['quantity'] * df_data['unit_price']
) * (1 - df_data['discount'])

# Rellenar cualquier NaT/NaN resultante de la operación con 0.0, asumiendo ventas no registradas.
df_data['TotalPriceCalculated'].fillna(0.0, inplace=True)

print("\nVerificación del cálculo de ventas totales (Top 5):")
print(df_data[['quantity', 'unit_price', 'discount', 'TotalPriceCalculated']].head())
#


Verificación del cálculo de ventas totales (Top 5):
   quantity  unit_price  discount  TotalPriceCalculated
0         7     44.2337       0.0             309.63590
1         7     62.5460       0.0             437.82200
2        24     79.0184       0.0            1896.44160
3        19     81.3167       0.2            1236.01384
4         9     79.9780       0.0             719.80200


/tmp/ipython-input-3121787520.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_data['TotalPriceCalculated'].fillna(0.0, inplace=True)


## 3. Detección y Marcado de Outliers (IQR)


In [15]:
# Consigna: Detecta outliers en TotalPriceCalculated utilizando el criterio del Rango Intercuartílico (IQR). Crea la columna IsOutlier.

# 1. Cálculo de Cuartiles y IQR
Q1 = df_data['TotalPriceCalculated'].quantile(0.25)
Q3 = df_data['TotalPriceCalculated'].quantile(0.75)
IQR = Q3 - Q1

# 2. Definición de Límites
LOWER_BOUND = Q1 - 1.5 * IQR
UPPER_BOUND = Q3 + 1.5 * IQR

# 3. Creación de la columna IsOutlier
df_data['IsOutlier'] = np.where(
    (df_data['TotalPriceCalculated'] < LOWER_BOUND) | (df_data['TotalPriceCalculated'] > UPPER_BOUND),
    1,  # Es un outlier
    0   # No es un outlier
)

# 4. Conteo de Outliers
outlier_count = df_data['IsOutlier'].sum()

print(f"\n--- Detección de Outliers (IQR) ---")
print(f"Q1 (25%): {Q1:,.2f}")
print(f"Q3 (75%): {Q3:,.2f}")
print(f"IQR: {IQR:,.2f}")
print(f"Límite Inferior (1.5*IQR): {LOWER_BOUND:,.2f}")
print(f"Límite Superior (1.5*IQR): {UPPER_BOUND:,.2f}")
print(f"Total de Outliers detectados: {outlier_count}")

# Justificación Técnica:
# Se eligió el método IQR porque es robusto y menos sensible a la presencia de valores extremos
# que podrían distorsionar la desviación estándar y la media.
# Esto es esencial en datos de ventas, que suelen tener una distribución sesgada a la derecha
# debido a las ventas excepcionalmente grandes.


--- Detección de Outliers (IQR) ---
Q1 (25%): 176.94
Q3 (75%): 982.16
IQR: 805.22
Límite Inferior (1.5*IQR): -1,030.89
Límite Superior (1.5*IQR): 2,189.99
Total de Outliers detectados: 48217


## 4. Análisis Temporal de Ventas


In [16]:
# Consigna 1 (Hora): Crea una nueva columna con la hora de la venta. Identifica la hora del día con más TotalPriceCalculated.

# 1. Creación de la columna 'SaleHour'
df_data['SaleHour'] = df_data['sales_date'].dt.hour

# 2. Identificación de la hora pico de ventas totales
hourly_sales = df_data.groupby('SaleHour')['TotalPriceCalculated'].sum().sort_values(ascending=False)
hour_peak = hourly_sales.index[0]
peak_sales_value = hourly_sales.iloc[0]

print(f"\n--- Análisis de Hora Pico ---")
print(f"Hora del día con mayor TotalPriceCalculated: {hour_peak}:00 (Ventas totales: {peak_sales_value:,.2f})")
#


--- Análisis de Hora Pico ---
Hora del día con mayor TotalPriceCalculated: 16.0:00 (Ventas totales: 179,014,421.24)


In [17]:
# Consigna 2 (Clasificación): Clasifica cada venta como 'Entre semana' o 'Fin de semana'. Compara el total de ventas.

# 1. Clasificación del día de la semana
# Lunes (0) a Viernes (4) es 'Entre semana'. Sábado (5) y Domingo (6) es 'Fin de semana'.
df_data['DayType'] = np.where(
    df_data['sales_date'].dt.dayofweek.isin([5, 6]), # 5: Sábado, 6: Domingo
    'Fin de semana',
    'Entre semana'
)

# 2. Comparación de Ventas Totales
sales_by_day_type = df_data.groupby('DayType')['TotalPriceCalculated'].sum().sort_values(ascending=False)
winner = sales_by_day_type.index[0]

print(f"\n--- Comparación de Ventas: Entre Semana vs. Fin de Semana ---")
print(sales_by_day_type.map('{:,.2f}'.format))

# Conclusión Fundamentada:
# Analizar si la concentración de ventas en 'Entre semana' vs. 'Fin de semana' justifica
# una reasignación de personal o inventario.
if winner == 'Entre semana':
    print(f"\nConclusión: La empresa vende más en '{winner}', sugiriendo que la mayoría de los clientes son compradores habituales o compras para negocio.")
else:
    print(f"\nConclusión: La empresa vende más en '{winner}', indicando una fuerte orientación al consumidor final o compras de 'stock' semanal.")


--- Comparación de Ventas: Entre Semana vs. Fin de Semana ---
DayType
Entre semana     3,123,404,728.48
Fin de semana    1,192,862,950.44
Name: TotalPriceCalculated, dtype: object

Conclusión: La empresa vende más en 'Entre semana', sugiriendo que la mayoría de los clientes son compradores habituales o compras para negocio.


## 5. Feature Engineering: Antigüedad del Empleado

In [18]:
# Consigna: Calcula (1) Edad del empleado al momento de la contratación y (2) Años de experiencia al momento de la venta.

# 1. Cálculo de la Edad al Contratar (HireDate - BirthDate)
# Se calcula la diferencia en días y se divide por 365.25 para obtener años con precisión.
df_data['AgeAtHire'] = (df_data['hire_date'] - df_data['birth_date']).dt.days / 365.25

# 2. Cálculo de la Experiencia al Vender (SalesDate - HireDate)
# Se calcula la diferencia en días y se divide por 365.25 para obtener años con precisión.
df_data['ExperienceYears'] = (df_data['sales_date'] - df_data['hire_date']).dt.days / 365.25

# Limpieza de valores inválidos (e.g., fechas faltantes o HireDate después de SalesDate)
df_data['AgeAtHire'].fillna(-1, inplace=True) # Usar -1 para indicar dato faltante/inválido
df_data['ExperienceYears'] = df_data['ExperienceYears'].apply(lambda x: x if x >= 0 else 0) # La experiencia no puede ser negativa

print(f"\n--- Verificación de Features de Empleados (Primeras 5 filas) ---")
print(df_data[['sales_date', 'birth_date', 'hire_date', 'AgeAtHire', 'ExperienceYears']].head())

# Justificación:
# El uso de años decimales (división por 365.25) en lugar de un cálculo simple de año permite
# una mayor precisión para modelos de Machine Learning, evitando la pérdida de información
# sobre el tiempo exacto transcurrido, una práctica clave en Feature Engineering temporal.

/tmp/ipython-input-2065911985.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_data['AgeAtHire'].fillna(-1, inplace=True) # Usar -1 para indicar dato faltante/inválido



--- Verificación de Features de Empleados (Primeras 5 filas) ---
               sales_date birth_date               hire_date  AgeAtHire  \
0 2018-02-05 07:38:25.430 1987-01-13 2013-06-22 13:20:18.080  26.439425   
1 2018-02-02 16:03:31.150 1951-07-07 2017-02-10 11:21:26.650  65.598905   
2 2018-05-03 19:31:56.880 1963-04-18 2011-12-12 10:43:52.940  48.651608   
3 2018-04-07 14:43:55.420 1956-12-13 2014-10-14 23:12:53.420  57.834360   
4 2018-02-12 15:37:03.940 1963-12-30 2012-07-23 15:02:12.640  48.563997   

   ExperienceYears  
0         4.621492  
1         0.977413  
2         6.390144  
3         3.477070  
4         5.557837  


## 6. Preparación del Dataset Definitivo para Modelado

In [20]:
# Consigna: Prepara un único dataset definitivo para modelado. Incluye features calculadas. Aplica transformaciones a variables categóricas y numéricas (si es necesario). La variable objetivo (TotalPriceCalculated) no debe transformarse.

# 1. Selección de Columnas Relevantes (Incluyendo Features Creadas)
# Se incluyen también features de otras tablas (clientes, ciudades, etc.) para enriquecer el modelo.

# Lista simplificada de columnas para el modelo
features_to_keep = [
    'TotalPriceCalculated',   # OBJETIVO (sin transformar)
    'quantity',               # Numérica
    'discount',               # Numérica
    'SaleHour',               # Categórica/Ordinal (transformada)
    'DayType',                # Categórica (transformada)
    'IsOutlier',              # Categórica/Binaria
    'AgeAtHire',              # Numérica (Feature Engineering)
    'ExperienceYears',        # Numérica (Feature Engineering)
    'ProductName',           # Categórica
    'category_name',          # Categórica
    'sales_person_id',        # Categórica (ID)
    'CustomerID',            # Categórica (ID)
    # Aquí se podrían añadir features de city_id, country_id, etc.
]

df_model = df_data[features_to_keep].copy()

# 2. Transformación de Variables Categóricas (One-Hot Encoding)
# Se utiliza One-Hot Encoding para categorías con bajo número de niveles (DayType, category_name)
# para evitar la asunción de orden (como en Label Encoding).

categorical_cols = ['DayType', 'category_name']
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# 3. Manejo Final de Nulos
# Tras la limpieza y feature engineering, cualquier remanente de NaN se imputa con la media o 0,
# dependiendo de la naturaleza de la columna.
numeric_cols = df_model.select_dtypes(include=np.number).columns
df_model[numeric_cols].fillna(df_model[numeric_cols].mean(), inplace=True)


print(f"\n--- Dataset Definitivo para Modelado ---")
print(f"Filas: {len(df_model)}, Columnas: {len(df_model.columns)}")
print(df_model.head())
print("\nTipos de datos del dataset final:")
print(df_model.dtypes)

# Justificación de las Transformaciones:
# 1. Feature Selection: Se eliminaron las columnas redundantes (e.g., 'total_price' original, fechas crudas)
#    y se conservaron las features de mayor valor predictivo.
# 2. One-Hot Encoding: Se aplicó a variables nominales ('DayType', 'category_name') para que el
#    modelo de ML pueda interpretar estas características sin asumir una relación ordinal
#    inexistente, creando un dataset totalmente numérico y listo para el entrenamiento.


--- Dataset Definitivo para Modelado ---
Filas: 6758125, Columnas: 21
   TotalPriceCalculated  quantity  discount  SaleHour  IsOutlier  AgeAtHire  \
0             309.63590         7       0.0       7.0          0  26.439425   
1             437.82200         7       0.0      16.0          0  65.598905   
2            1896.44160        24       0.0      19.0          0  48.651608   
3            1236.01384        19       0.2      14.0          0  57.834360   
4             719.80200         9       0.0      15.0          0  48.563997   

   ExperienceYears                 ProductName  sales_person_id  CustomerID  \
0         4.621492            Vaccum Bag 10x13                6       27039   
1         0.977413                    Sardines               16       25011   
2         6.390144     Crab - Imitation Flakes               13       94024   
3         3.477070  Smirnoff Green Apple Twist                8       73966   
4         5.557837         Coffee - Dark Roast             

/tmp/ipython-input-3619369719.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[numeric_cols].fillna(df_model[numeric_cols].mean(), inplace=True)
